In [1]:
import sklearn.gaussian_process as gp
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import matplotlib
import time

In [2]:
from gp_kernels import *
from gp_priors import *
from gp_plots import *

In [ ]:
generate_priors()

In [3]:
plot_gp_prior(4, n=20)

[[1.00000000e+00 8.70659634e-01 5.74637070e-01 2.87498569e-01
  1.09037166e-01 3.13480292e-02 6.83191964e-03 1.12868324e-03
  1.41350784e-04 1.34190319e-05 9.65697197e-07 5.26814324e-08
  2.17856739e-09 6.82937941e-11 1.62288708e-12 2.92342653e-14
  3.99202163e-16 4.13228632e-18 3.24253555e-20 1.92874985e-22]
 [8.70659634e-01 1.00000000e+00 8.70659634e-01 5.74637070e-01
  2.87498569e-01 1.09037166e-01 3.13480292e-02 6.83191964e-03
  1.12868324e-03 1.41350784e-04 1.34190319e-05 9.65697197e-07
  5.26814324e-08 2.17856739e-09 6.82937941e-11 1.62288708e-12
  2.92342653e-14 3.99202163e-16 4.13228632e-18 3.24253555e-20]
 [5.74637070e-01 8.70659634e-01 1.00000000e+00 8.70659634e-01
  5.74637070e-01 2.87498569e-01 1.09037166e-01 3.13480292e-02
  6.83191964e-03 1.12868324e-03 1.41350784e-04 1.34190319e-05
  9.65697197e-07 5.26814324e-08 2.17856739e-09 6.82937941e-11
  1.62288708e-12 2.92342653e-14 3.99202163e-16 4.13228632e-18]
 [2.87498569e-01 5.74637070e-01 8.70659634e-01 1.00000000e+00
  8.7

NameError: name 'k' is not defined